### Import the libraries

In [ ]:
from astropy.time import Time
from astroquery.jplhorizons import Horizons

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import requests
import time

In [ ]:
# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

_________________________________________________________

### Read the catalog

In [ ]:
df = pd.read_csv('GaiaDR3-result.csv', dtype={'number_mp': 'int'})

In [ ]:
# This catalog has many lines with jd values but empty magnitudes. We don't need them.
df = df.dropna()

### List of designations

In [ ]:
ids = df['number_mp'].drop_duplicates().to_numpy()

In [ ]:
len(ids)

### Avoid repeating asteorids

In [ ]:
# # Specify the folder path where your CSV files are located
folder_path = '/home/milagros/Documents/Gaia-files'

# # Initialize an empty list to store the formatted asteroid numbers
formatted_asteroid_numbers = []

# # Define a regular expression pattern to match the filenames
pattern = r'GAIADR2-asteroid(\d+)\.csv'

# # List all files in the folder
files = os.listdir(folder_path)

# # Iterate through the files and extract the numbers
for filename in files:
    match = re.match(pattern, filename)
    if match:
        number = int(match.group(1))
       
        formatted_asteroid_numbers.append(number)

In [ ]:
df = df[~df['number_mp'].isin(formatted_asteroid_numbers)]

### Dataframe

In [ ]:
# We need to perform some calculations in order to obtain the desired format for the table.

jd = df['epoch_utc']+2455197.5
df.insert(2, "jd", jd, True)

m = -2.5*np.log10(df['g_flux']) + 25.6884
df.insert(3, "m", m, True)

err_m = np.sqrt((-2.5*(1/np.log(10))*(df['g_flux_error']/df['g_flux']))**2 + (0.0018)**2)
df.insert(4, "err_m", err_m, True)

In [ ]:
df = df.rename(columns={'number_mp':'idAsteroid', 'g_flux':'flux', 'g_flux_error':'err_flux'})

In [ ]:
df = df[['idAsteroid', 'jd', 'flux', 'err_flux', 'm', 'err_m']]

In [ ]:
# ----------------------------------------------------------------------------------------------------------------------
#Add constant columns
df.insert(6, 'Source', 'Gaia')
df.insert(7, 'TypePhotometry', 'relative')
df.insert(8, 'LTcorrected', '0')
df.insert(9, 'Filter', 'G')
# ----------------------------------------------------------------------------------------------------------------------
#Add ephemeris columns
df.insert(10, 'ElongFlag', '')
df.insert(11, 'alpha', '')
df.insert(12, 'r', '')
df.insert(13, 'delta', '')
df.insert(14, 'x_obs', '')
df.insert(15, 'y_obs', '')
df.insert(16, 'z_obs', '')
df.insert(17, 'x_sun', '')
df.insert(18, 'y_sun', '')
df.insert(19, 'z_sun', '')
df.insert(20, 'LT_obs', '')

_______________________________________________________________________-

### Splitting dataframe

We have a dataframe with all the asteroids. For the ephemeris step, we need to proceed asteroid by asteroid.

In [ ]:
sample = df

In [ ]:
ids_sample = sample['idAsteroid'].drop_duplicates()
ids_sample = ids_sample.to_numpy()
len = ids_sample.size

In [ ]:
# Function to split DataFrame based on the changing value in the 'target_column'
def split_dataframe_on_value_change(df, column_name):
    df_list = []
    prev_value = None
    temp_df = None

    for index, row in df.iterrows():
        current_value = row[column_name]

        if prev_value is None or current_value != prev_value:
            if temp_df is not None:
                df_list.append(temp_df)
            temp_df = pd.DataFrame(columns=df.columns)

        temp_df = pd.concat([temp_df, row.to_frame().T], ignore_index=True)
        prev_value = current_value

    if temp_df is not None:
        df_list.append(temp_df)

    return df_list

In [ ]:
# Split the DataFrame based on the changing value in the 'target_column'
print('Splitting')
resulting_dataframes = split_dataframe_on_value_change(sample, 'idAsteroid')
print('Splitted')

### Ephemeris

In [ ]:
# Ephemeris
for idx, df in enumerate(resulting_dataframes):
    ids = df['idAsteroid'].drop_duplicates().to_numpy()[0]
    jd = df['jd'].to_numpy()

    # If the list we pass to Horizons contains more than 50 elements, it throws an error.
    if (jd.size <= 50):
        obj = Horizons(id=ids, location='@Gaia',
                epochs=jd, id_type='smallbody')
    
        #Ephemeris query
        eph = obj.ephemerides()
        elong_flag = eph.columns['elongFlag']
        alpha = eph.columns['alpha']
        delta = eph.columns['delta']
        r = eph.columns['r']
        
        #Vectors query
        vec = obj.vectors()
        x_obs = vec.columns['x']
        y_obs = vec.columns['y']
        z_obs = vec.columns['z']
        LT_obs = vec.columns['lighttime']
    
        #Query Sun-----------------------------------------
        jd_tdb = Time(jd.tolist(), format='jd', scale='utc').tdb.value
        obj_sun = Horizons(id=ids, location='500@10',
                epochs=jd_tdb, id_type='smallbody')
        vec_sun = obj.vectors()
        x_sun = vec_sun.columns['x']
        y_sun = vec_sun.columns['y']
        z_sun = vec_sun.columns['z']  
    
        #Put in dataframe
        #df.loc[idx,'Desig'] = desig[0]
        df['ElongFlag'] = elong_flag
        df['alpha'] = alpha
        df['r'] = r
        df['delta'] = delta
        df['x_obs'] = x_obs
        df['y_obs'] = y_obs
        df['z_obs'] = z_obs
        df['x_sun'] = x_sun
        df['y_sun'] = y_sun
        df['z_sun'] = z_sun
        df['LT_obs'] = LT_obs
        
        m_red = df['m'] - 2.5*np.log10(df['r']*df['delta'])
        df.insert(6, "m_red", m_red, True)
        
        df.to_csv('/home/milagros/Documents/Gaia-files/GAIADR3-asteroid'+str(ids)+'.csv', index=False) 
        
        del obj
        del eph
        del obj_sun
        del vec_sun
        gc.collect()
        print(ids, 'small done')

    # To process lists of more than 50 elements, we split the dataframe and work part by part. At the end we merge everything into a single table.
    if (jd.size > 50):
        split_dataframes = np.array_split(df, 5)
        calculated_dataframes = []
        for i in split_dataframes:
            obj = Horizons(id=ids, location='@Gaia',
            epochs=i.jd.to_numpy(), id_type='smallbody')
    
            #Ephemeris query
            eph = obj.ephemerides()
            elong_flag = eph.columns['elongFlag']
            alpha = eph.columns['alpha']
            delta = eph.columns['delta']
            r = eph.columns['r']
            
            #Vectors query
            vec = obj.vectors()
            x_obs = vec.columns['x']
            y_obs = vec.columns['y']
            z_obs = vec.columns['z']
            LT_obs = vec.columns['lighttime']
        
            #Query Sun-----------------------------------------
            jd_tdb = Time(i.jd.tolist(), format='jd', scale='utc').tdb.value
            obj_sun = Horizons(id=ids, location='500@10',
                    epochs=jd_tdb, id_type='smallbody')
            vec_sun = obj.vectors()
            x_sun = vec_sun.columns['x']
            y_sun = vec_sun.columns['y']
            z_sun = vec_sun.columns['z']  
        
            #Put in dataframe
            #df.loc[idx,'Desig'] = desig[0]
            i['ElongFlag'] = elong_flag
            i['alpha'] = alpha
            i['r'] = r
            i['delta'] = delta
            i['x_obs'] = x_obs
            i['y_obs'] = y_obs
            i['z_obs'] = z_obs
            i['x_sun'] = x_sun
            i['y_sun'] = y_sun
            i['z_sun'] = z_sun
            i['LT_obs'] = LT_obs
            
            m_red = i['m'] - 2.5*np.log10(i['r']*i['delta'])
            i['m_red'] = m_red
            
            calculated_dataframes.append(i)
            del obj
            del eph
            del obj_sun
            del vec_sun
            gc.collect()
            
        merged_df = pd.concat(calculated_dataframes, ignore_index=True)
        merged_df['ElongFlag'] = merged_df['ElongFlag'].to_numpy()
        merged_df['alpha'] = merged_df['alpha'].to_numpy()
        merged_df['r'] = merged_df['r'].to_numpy()
        merged_df['delta'] = merged_df['delta'].to_numpy()
        merged_df['m_red'] = merged_df['m_red'].to_numpy()
        merged_df['x_obs'] = merged_df['x_obs'].to_numpy()
        merged_df['y_obs'] = merged_df['y_obs'].to_numpy()
        merged_df['z_obs'] = merged_df['z_obs'].to_numpy()
        merged_df['x_sun'] = merged_df['x_sun'].to_numpy()
        merged_df['y_sun'] = merged_df['y_sun'].to_numpy()
        merged_df['z_sun'] = merged_df['z_sun'].to_numpy()
        merged_df['LT_obs'] = merged_df['LT_obs'].to_numpy()
        
        merged_df.to_csv('/home/milagros/Documents/Gaia-files/GAIADR3-asteroid'+str(ids)+'.csv', index=False)
        print(ids, 'big done')
    

The end :)